Load packages

In [ ]:
#loading packages
push!(LOAD_PATH, "src");
import Base: @kwdef
using Pkg
Pkg.instantiate()
using Test
using LifeInsuranceDataModel

using BitemporalPostgres
using SearchLight
using SearchLightPostgreSQL
using TimeZones
using ToStruct
using JSON

purging the data model entirely - empty the schema

In [ ]:
if (haskey(ENV, "GITPOD_REPO_ROOT"))
    run(```psql -f sqlsnippets/droptables.sql```)
else
    println("execute this in Your REPL")
    println("""run(```sudo -u postgres psql -f sqlsnippets/droptables.sql```)
        
        """)
end


Loading the data model- Create tables, constraints etc. and load Roles

loading inverses of the role tables to provide role descriptions in object creation,  for instance like in: "ref_role=cpRole["Policy Holder"]

In [ ]:
LifeInsuranceDataModel.load_model()

cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end


Or just connect to an existing model

In [ ]:
LifeInsuranceDataModel.connect()

Create a Partner

In [ ]:
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1")
w = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)

Partner1 = p.id.value

Create tariffs

In [15]:
# create Tariffs
function create_tariff(dsc,mt)
    t = LifeInsuranceDataModel.Tariff()
    tr = LifeInsuranceDataModel.TariffRevision(description=dsc,mortality_table=mt)
    w = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    create_entity!(w)
    create_component!(t, tr, w)
    commit_workflow!(w)
    t.id.value
end

LifeRiskTariff = create_tariff("Life Risk Insurance", "1980 CET - Male Nonsmoker, ANB")
TerminalIllnessTariff = create_tariff("Terminal Illness", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")
OccupationalDisabilityTariff = create_tariff("Occupational Disability", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")
ProfitParticipationTariff = create_tariff("Profit participation", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")
LifeRiskTariff2 = create_tariff("Life Risk Insurance", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")

find(TariffRevision)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 3 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'3' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 6 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/Searc

┌ Info: INSERT INTO tariffrevisions ( "ref_component", "ref_validfrom", "ref_invalidfrom", "description", "mortality_table" ) VALUES ( E'1', E'6', E'9223372036854775807', E'Life Risk Insurance', E'1980 CET - Male Nonsmoker, ANB' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (1,1,6,9223372036854775807,,"Life Risk Insurance","1980 CET - Male Nonsmoker, ANB")
┌ Info: SELECT "tariffrevisions"."id" AS "tariffrevisions_id", "tariffrevisions"."ref_component" AS "tariffrevisions_ref_component", "tariffrevisions"."ref_validfrom" AS "tariffrevisions_ref_validfrom", "tariffrevisions"."ref_invalidfrom" AS "tariffrevisions_ref_invalidfrom", "tariffrevisions"."description" AS "tariffrevisions_description", "tariffrevisions"."mortality_table" AS "tariffrevisions_mortality_table" FROM "tariffrevisions" WHERE "id" = 1 ORDER BY tariffrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLig

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'6' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.juli

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'7' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.juli

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'8' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.juli

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'9' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.juli

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'10' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.jul

5-element Vector{TariffRevision}:
 TariffRevision
| KEY                     | VALUE                          |
|-------------------------|--------------------------------|
| description::String     | Life Risk Insurance            |
| id::DbId                | 1                              |
| mortality_table::String | 1980 CET - Male Nonsmoker, ANB |
| ref_component::DbId     | 1                              |
| ref_invalidfrom::DbId   | 9223372036854775807            |
| ref_validfrom::DbId     | 6                              |

 TariffRevision
| KEY                     | VALUE                                                                |
|-------------------------|----------------------------------------------------------------------|
| description::String     | Terminal Illness                                                     |
| id::DbId                | 2                                                                    |
| mortality_table::String | 2001 VBT Residual Sta

In [ ]:
find(Tariff,SQLWhereExpression("id=?",ProfitParticipationTariff))
find(Tariff, SQLWhereExpression("id=?", TerminalIllnessTariff))

Create Product

In [ ]:
p = Product()
pr = ProductRevision(description="Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(ref_tariff=LifeRiskTariff, ref_role=ppRole["Main Coverage - Life"], description="Main Coverage - Life")

pp2 = ProductPart()
ppr2 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Lif Risk")

w0 = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
commit_workflow!(w0)

LifeRiskProduct = p.id.value
println(LifeRiskProduct)

p = Product()
pr = ProductRevision(description="Life Risk - Terminal Illness")

pp = ProductPart()
ppr = ProductPartRevision(ref_tariff=LifeRiskTariff, ref_role=ppRole["Main Coverage - Life"], description="Main Coverage - Life")

pp2 = ProductPart()
ppr2 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Life Risk")

pp3 = ProductPart()
ppr3 = ProductPartRevision(ref_tariff=TerminalIllnessTariff, ref_role=ppRole["Supplementary Coverage - Terminal Illness"], description="additional cover Terminal Illness")

pp4 = ProductPart()
ppr4 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Terminal Illness")

pp5 = ProductPart()
ppr5 = ProductPartRevision(ref_tariff=OccupationalDisabilityTariff, ref_role=ppRole["Supplementary Coverage - Occupational Disablity"], description="additional cover Occupational Disablity")

pp6 = ProductPart()
ppr6 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Occ.Disablity")



w0 = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
create_subcomponent!(p, pp3, ppr3, w0)
create_subcomponent!(p, pp4, ppr4, w0)
create_subcomponent!(p, pp5, ppr5, w0)
create_subcomponent!(p, pp6, ppr6, w0)
commit_workflow!(w0)

LifeRiskTIODProduct = p.id.value
println(LifeRiskTIODProduct)

Create contract

In [ ]:
    w1 = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    
    create_entity!(w1)
    c = Contract()
    cr = ContractRevision(description="contract creation properties")
    create_component!(c, cr, w1)
    
    cpr = ContractPartnerRef(ref_super=c.id)
    cprr = ContractPartnerRefRevision(ref_partner=Partner1, ref_role=cpRole["Policy Holder"], description="policiyholder ref properties")
    create_subcomponent!(c, cpr, cprr, w1)
    # pi 1
    cpi = ProductItem(ref_super=c.id)
    cpir = ProductItemRevision(position=1, ref_product=LifeRiskTIODProduct, description="from contract creation")
    create_subcomponent!(c, cpi, cpir, w1)
    # pi 1 ti 1 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=LifeRiskTariff, ref_role=titrRole["Main Coverage - Life"], description="Life Risk tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 1 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 2 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profit participation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 2 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 3 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=TerminalIllnessTariff, ref_role=titrRole["Supplementary Coverage - Terminal Illness"], description="Terminal Illness tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 3 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 4 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profitparticipation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 4 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    
    # pi 1 ti 5 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Supplementary Coverage - Occupational Disablity"], description="Occupational Disablity tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 5 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 6 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profit Participation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 6 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    
    
    
    commit_workflow!(w1)

update Contract yellow

In [ ]:
cr1 = ContractRevision(ref_component=c.id, description="contract 1, 2nd mutation")
w2 = Workflow(
    ref_history=w1.ref_history,
    tsw_validfrom=ZonedDateTime(2016, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w2)
update_component!(cr, cr1, w2)
commit_workflow!(w2)

   update Contract red

In [ ]:

cr2 = ContractRevision(ref_component=c.id, description="contract 1, 3rd mutation retrospective")
w3 = Workflow(
    ref_history=w2.ref_history,
    tsw_validfrom=ZonedDateTime(2015, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w3)
update_component!(cr1, cr2, w3)


commit_workflow!(w3)


In [ ]:
w4 = Workflow(
    ref_history=w2.ref_history,
    tsw_validfrom=ZonedDateTime(2018, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w4)
cr3 = ContractRevision(ref_component=c.id, description="contract 1, 4th mutation")
update_component!(cr2, cr3, w4)

# pi 2
LifeRiskTIODProduct = find(Product, SQLWhereExpression("id=?", 2))[1].id.value
Partner1
PartnerRole = tiprRole["Insured Person"]

cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(position=2, ref_product=LifeRiskTIODProduct, description="from contract 4th mutation")
create_subcomponent!(c, cpi, cpir, w4)

println("vor neuem Teil")



map(find(ProductPart, SQLWhereExpression("ref_super=?", LifeRiskTIODProduct))) do pp
    println(pp.id.value)
    map(find(ProductPartRevision, SQLWhereExpression("ref_component=?", pp.id.value))) do ppr
        println(ppr.description)
        ti=TariffItem(ref_super=cpi.id)
        tir = TariffItemRevision(ref_role=ppr.ref_role, ref_tariff=ppr.ref_tariff, description=ppr.description)
        create_subcomponent!(cpi, ti, tir, w4)
        tip = TariffItemPartnerRef(ref_super=cpi.id)
        tipr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=PartnerRole)
        create_subcomponent!(cpi, tip, tipr, w4)
        println(tir)
        println(tipr)
    end
end
println("vor rollback")
# rollback_workflow!(w4)
commit_workflow!(w4)